In [8]:
pip install gdown pydrive tqdm tensorflow opencv-python


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:

import os
import zipfile
import shutil
import traceback
from tqdm import tqdm
import gdown
import tensorflow as tf
import cv2
import numpy as np
import random
import glob
from tensorflow.keras.applications import Xception
from tensorflow.keras import layers, models, optimizers



In [10]:
FILE_ID = '1Mu69hp0vWC_a1GqnP1u3LGLqrSfKRNoy'   
OUTPUT_ZIP = 'Deepfake.zip'
LOCAL_DATA_ROOT = './deepfake_dataset_local'
UNZIP = True
FRAMES_PER_VIDEO = 6
FRAME_SIZE = (160, 160)


In [11]:
if not os.path.exists(OUTPUT_ZIP):
    print("📥 Downloading dataset from Google Drive...")
    gdown.download(f"https://drive.google.com/uc?id={FILE_ID}", OUTPUT_ZIP, quiet=False)
else:
    print("✅ Dataset ZIP already exists.")


📥 Downloading dataset from Google Drive...


Downloading...
From (original): https://drive.google.com/uc?id=1Mu69hp0vWC_a1GqnP1u3LGLqrSfKRNoy
From (redirected): https://drive.google.com/uc?id=1Mu69hp0vWC_a1GqnP1u3LGLqrSfKRNoy&confirm=t&uuid=e22ad379-fe9c-4b0e-94e4-197cbeeded03
To: c:\Users\shail\Documents\T87-Deepfake-Video-Detection-System\Deepfake.zip
100%|██████████| 4.02G/4.02G [08:07<00:00, 8.25MB/s]


In [12]:
if UNZIP:
    print("📦 Extracting dataset...")
    os.makedirs(LOCAL_DATA_ROOT, exist_ok=True)
    try:
        with zipfile.ZipFile(OUTPUT_ZIP, 'r') as z:
            for member in tqdm(z.namelist(), desc="Extracting"):
                try:
                    z.extract(member, LOCAL_DATA_ROOT)
                except Exception as e:
                    print("Error extracting", member, ":", e)
        print("✅ Extraction complete:", LOCAL_DATA_ROOT)
    except Exception:
        traceback.print_exc()
        raise RuntimeError("Failed to unzip dataset.")


📦 Extracting dataset...


Extracting: 100%|██████████| 140003/140003 [02:29<00:00, 935.99it/s] 

✅ Extraction complete: ./deepfake_dataset_local


In [13]:
def find_dataset_root(base):
    for root, dirs, files in os.walk(base):
        if {'train','valid','test'}.issubset(set(dirs)):
            return root
        if {'real','fake'}.issubset(set(dirs)):
            return root
    return None

DATA_ROOT = find_dataset_root(LOCAL_DATA_ROOT)
if DATA_ROOT is None:
    print("Could not auto-detect dataset root. Inspect:", LOCAL_DATA_ROOT)
    print("Contents:", os.listdir(LOCAL_DATA_ROOT))
else:
    print("Detected dataset root:", DATA_ROOT)
    
for root, dirs, files in os.walk(DATA_ROOT):
    depth = root.replace(DATA_ROOT, '').count(os.sep)
    indent = ' ' * 4 * depth
    print(f"{indent}{os.path.basename(root)}/ (files: {len(files)})")
    if depth >= 2:
        continue


Detected dataset root: ./deepfake_dataset_local\real_vs_fake\real-vs-fake
real-vs-fake/ (files: 0)
    test/ (files: 0)
        fake/ (files: 10000)
        real/ (files: 10000)
    train/ (files: 0)
        fake/ (files: 50000)
        real/ (files: 50000)
    valid/ (files: 0)
        fake/ (files: 10000)
        real/ (files: 10000)


In [14]:
def ensure_splits(root, ratio=(0.8, 0.1, 0.1)):
    splits = ['train','valid','test']
    classes = []

    if os.path.exists(os.path.join(root, 'train')):
        print("train/valid/test already exist. Skipping split.")
        return root

    for cls in ['real','fake']:
        if os.path.exists(os.path.join(root, cls)):
            classes.append(cls)

    if not classes:
        raise RuntimeError("No 'real'/'fake' folders found.")

    for cls in classes:
        cls_path = os.path.join(root, cls)
        items = [f for f in os.listdir(cls_path) if not f.startswith('.')]
        random.shuffle(items)
        n = len(items)
        n_train = int(ratio[0]*n)
        n_valid = int(ratio[1]*n)

        for split in splits:
            os.makedirs(os.path.join(root, split, cls), exist_ok=True)

        for i, f in enumerate(items):
            src = os.path.join(cls_path, f)
            if i < n_train:
                dst = os.path.join(root, 'train', cls, f)
            elif i < n_train + n_valid:
                dst = os.path.join(root, 'valid', cls, f)
            else:
                dst = os.path.join(root, 'test', cls, f)
            shutil.move(src, dst)

        try: os.rmdir(cls_path)
        except OSError: pass

    print("✅ Created train/valid/test splits.")
    return root

ROOT = ensure_splits(DATA_ROOT)
print("Final ROOT:", ROOT)

# Verify
for split in ['train','valid','test']:
    for cls in ['real','fake']:
        p = os.path.join(ROOT, split, cls)
        print(f"{split}/{cls} → {len(os.listdir(p)) if os.path.exists(p) else 0} files")



train/valid/test already exist. Skipping split.
Final ROOT: ./deepfake_dataset_local\real_vs_fake\real-vs-fake
train/real → 50000 files
train/fake → 50000 files
valid/real → 10000 files
valid/fake → 10000 files
test/real → 10000 files
test/fake → 10000 files


In [16]:
OUT_FRAMES_DIR = './frames_dataset'
os.makedirs(OUT_FRAMES_DIR, exist_ok=True)

def extract_frames_from_video(video_path, out_dir, n_frames=FRAMES_PER_VIDEO, resize=FRAME_SIZE):
    os.makedirs(out_dir, exist_ok=True)
    cap = cv2.VideoCapture(video_path)
    total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if total <= 0:
        cap.release()
        return 0
    indices = list(map(int, np.linspace(0, max(total-1,0), n_frames)))
    saved = 0
    for idx in indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ret, frame = cap.read()
        if not ret:
            continue
        frame = cv2.resize(frame, resize)
        cv2.imwrite(os.path.join(out_dir, f'frame_{saved:03d}.jpg'), frame)
        saved += 1
    cap.release()
    return saved

for split in ['train','valid','test']:
    for cls in ['real','fake']:
        in_dir = os.path.join(ROOT, split, cls)
        out_split_cls = os.path.join(OUT_FRAMES_DIR, split, cls)
        os.makedirs(out_split_cls, exist_ok=True)
        videos = [f for f in os.listdir(in_dir) if os.path.isfile(os.path.join(in_dir, f))]
        print(f"Processing {split}/{cls}: {len(videos)} videos")
        for v in tqdm(videos, desc=f"{split}/{cls}"):
            vname = os.path.splitext(v)[0]
            out_folder = os.path.join(out_split_cls, vname)
            if os.path.exists(out_folder): continue
            try:
                extract_frames_from_video(os.path.join(in_dir, v), out_folder)
            except Exception as e:
                print("Error:", v, e)

print("Frame extraction complete →", OUT_FRAMES_DIR)


Processing train/real: 50000 videos


train/real: 100%|██████████| 50000/50000 [03:42<00:00, 224.33it/s] 


Processing train/fake: 50000 videos


train/fake: 100%|██████████| 50000/50000 [04:19<00:00, 192.98it/s]


Processing valid/real: 10000 videos


...

Processing test/real: 10000 videos


test/real: 100%|██████████| 10000/10000 [00:49<00:00, 202.63it/s]


Processing test/fake: 10000 videos


test/fake: 100%|██████████| 10000/10000 [00:47<00:00, 212.21it/s]

Frame extraction complete → ./frames_dataset


In [20]:

from tensorflow.keras.applications.xception import preprocess_input

class VideoSequence(tf.keras.utils.Sequence):
    def __init__(self, list_dirs, batch_size=4, frames=FRAMES_PER_VIDEO, shuffle=True):
        self.list_dirs = list_dirs
        self.batch_size = batch_size
        self.frames = frames
        self.shuffle = shuffle
        self.indexes = np.arange(len(self.list_dirs))
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.list_dirs) / self.batch_size))

    def __getitem__(self, idx):
        batch_idxs = self.indexes[idx*self.batch_size:(idx+1)*self.batch_size]
        dirs = [self.list_dirs[i] for i in batch_idxs]
        X = np.zeros((len(dirs), self.frames, FRAME_SIZE[0], FRAME_SIZE[1], 3), dtype=np.float32)
        y = np.zeros((len(dirs),), dtype=np.int32)

        for i, d in enumerate(dirs):
            frames_files = sorted(glob.glob(os.path.join(d, '*.jpg')))
            if len(frames_files) < self.frames:
                frames_files = frames_files + [frames_files[-1]]*(self.frames - len(frames_files))
            frames_files = frames_files[:self.frames]
            arrs = []
            for f in frames_files:
                img = tf.keras.preprocessing.image.load_img(f, target_size=FRAME_SIZE)
                ar = tf.keras.preprocessing.image.img_to_array(img)
                ar = preprocess_input(ar)
                arrs.append(ar)
            X[i] = np.stack(arrs)
            y[i] = 0 if '/real/' in d else 1
        return X, tf.keras.utils.to_categorical(y, num_classes=2)

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indexes)

def collect_dirs(frames_root, split):
    out = []
    for cls in ['real','fake']:
        base = os.path.join(frames_root, split, cls)
        if not os.path.exists(base): continue
        for v in os.listdir(base):
            if os.path.isdir(os.path.join(base, v)):
                out.append(os.path.join(base, v))
    return out

train_dirs = collect_dirs(OUT_FRAMES_DIR, 'train')
valid_dirs = collect_dirs(OUT_FRAMES_DIR, 'valid')
test_dirs = collect_dirs(OUT_FRAMES_DIR, 'test')

print("train:", len(train_dirs), "valid:", len(valid_dirs), "test:", len(test_dirs))


train: 100000 valid: 20000 test: 20000


In [21]:
def build_fast_xception_model(frames=FRAMES_PER_VIDEO):
    base = Xception(weights='imagenet', include_top=False, pooling='avg', input_shape=(FRAME_SIZE[0], FRAME_SIZE[1], 3))
    base.trainable = False
    inp = layers.Input(shape=(frames, FRAME_SIZE[0], FRAME_SIZE[1], 3))
    x = layers.TimeDistributed(base)(inp)
    x = layers.Conv1D(64, 3, padding='same', activation='relu')(x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    out = layers.Dense(2, activation='softmax')(x)
    model = models.Model(inputs=inp, outputs=out)
    return model

model = build_fast_xception_model()



In [22]:
model.compile(optimizer=optimizers.Adam(1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 6, 160, 160, 3) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_1              │ (None, 6, 2048)        │    20,861,480 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 6, 64)          │       393,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_1      │ (None, 64)             │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,259,050 (81.10 MB)

 Trainable params: 397,570 (1.52 MB)

 Non-trainable params: 20,861,480 (79.58 MB)